In [1]:
# B
# import packges
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
import nltk
from nltk.corpus import stopwords
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity
import time

import missingno as msno
import math

english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']


In [2]:
filename = 'data/prep.csv'
df = pd.read_csv(filename, index_col='Unnamed: 0')
df.head()
df = df.sample(n = 1000)
# read df
df = df[['asin','reviewText']]



In [4]:
start = time.time()

# apply tokenization
df_a['tokens'] =  df["reviewText"].apply(word_tokenize)

# apply remove stopwords 
df_a["removeStopwords"] = df_a["tokens"].apply(lambda x: ([word for word in x if word not in (stopwords.words())]))

# remove punkt

df_a["removePunk"] = df_a["removeStopwords"].apply(lambda x: ([word for word in x if word not in (english_punctuations)]))

# lower case
df_a["lowStr"] = df_a["removePunk"].apply(lambda x: ([word.lower() for word in x]))

#Lemmatization

wnl = nltk.WordNetLemmatizer()
df_a["lem"] = df_a["lowStr"].apply(lambda x: ([wnl.lemmatize(word) for word in x]))

# clean df
df_b = df_a[['asin', 'lem']]
df_b["lem"] = df_b["lem"].apply(lambda x: ','.join(map(str, x)))

print(time.time() - start)

/var/folders/rl/7xl6y31j48d79l6t9q0p0z4w0000gn/T/ipykernel_76392/738742511.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a['tokens'] =  df_a["reviewText"].apply(word_tokenize)
/var/folders/rl/7xl6y31j48d79l6t9q0p0z4w0000gn/T/ipykernel_76392/738742511.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a["removeStopwords"] = df_a["tokens"].apply(lambda x: ([word for word in x if word not in (stopwords.words())]))


77.04782223701477


/var/folders/rl/7xl6y31j48d79l6t9q0p0z4w0000gn/T/ipykernel_76392/738742511.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b["lem"] = df_b["lem"].apply(lambda x: ','.join(map(str, x)))


In [5]:
# functions 
def create_corpus(df):
    corpus = []
    for i in df:
        corpus.append(i)
    return corpus

# calculate cosine_similarity
def cs(i,j):
  doc1 = df.iloc[i]/df.max(axis=0)
  doc2 = df.iloc[j]/df.max(axis=0)

  docAB = doc1.mul(doc2, axis=0)
  numerator = docAB.sum() # inner prodoct

  powdf = pd.DataFrame([2]*col)
  powdf = powdf.T
  doc1square = doc1.pow(powdf)
  doc2square = doc2.pow(powdf)
  sum1 = doc1square.sum(axis = 1)
  sum2 = doc2square.sum(axis = 1)
  sqrtsum1 = math.sqrt(sum1)
  sqrtsum2 = math.sqrt(sum2)
  denominator = sqrtsum1 * sqrtsum2 # product of A vec and B vec
  if denominator == 0:
    cosSim = 0
  else:
    cosSim= numerator/denominator
  # print('cosSim(doc',i+1,',','doc',j+1,') is')
  # print(" ") 
  return cosSim


In [6]:
# df_a["tokens"]
df_a.head()

,asin,reviewText,tokens,removeStopwords,removePunk,lowStr,lem
17167,B003U2UCTO,This mat has a great printed design. Its size ...,"[This, mat, has, a, great, printed, design, .,...","[This, great, printed, design, ., Its, size, l...","[This, great, printed, design, Its, size, long...","[this, great, printed, design, its, size, long...","[this, great, printed, design, it, size, longe..."
603219,B0018LA0UK,Good fit for my Winchester model 11,"[Good, fit, for, my, Winchester, model, 11]","[Good, fit, Winchester, model, 11]","[Good, fit, Winchester, model, 11]","[good, fit, winchester, model, 11]","[good, fit, winchester, model, 11]"
434212,B0012YUET0,these have kick there cool cuz if you hit any ...,"[these, have, kick, there, cool, cuz, if, you,...","[kick, cool, cuz, hit, solid, walls, metal, me...","[kick, cool, cuz, hit, solid, walls, metal, me...","[kick, cool, cuz, hit, solid, walls, metal, me...","[kick, cool, cuz, hit, solid, wall, metal, mel..."
82161,B00438RI7S,just as described,"[just, as, described]",[described],[described],[described],[described]
488991,B00G8QGHL6,Great tires,"[Great, tires]","[Great, tires]","[Great, tires]","[great, tires]","[great, tire]"


In [7]:
df_a.shape

(1000, 7)

In [8]:
df_b.head()

,asin,lem
17167,B003U2UCTO,"this,great,printed,design,it,size,longer,mat,i..."
603219,B0018LA0UK,"good,fit,winchester,model,11"
434212,B0012YUET0,"kick,cool,cuz,hit,solid,wall,metal,melted,dent..."
82161,B00438RI7S,described
488991,B00G8QGHL6,"great,tire"


In [9]:
corpusA = create_corpus(df_b["lem"])

In [10]:
corpusA

['this,great,printed,design,it,size,longer,mat,i,owned,easier,head,cushioned,position,i,highly,recommend,product',
 'good,fit,winchester,model,11',
 'kick,cool,cuz,hit,solid,wall,metal,melted,dent,quarter',
 'described',
 'great,tire',
 'excellent',
 "it,'s,product,i,wish,pocket,bit,larger,-,i,stretch,'s,difficult,foot,loop,sneaker",
 'recommended,ginger,cook',
 'fantastic,sock,comfy,form,fitting,cute,color',
 'easy,put,sturdy',
 "fit,finish,top,notch,fit,rail,ctr,buttstock,swivel,mount,perfectly,american,made,half,price,``,brand,'','s,highly,recommended",
 "83,buck,tent,free,shipping,n't,beat,i,'ve,backyard,camping,3,time,it,'s,easy,set-up,huge,set-up,it,'s,breeze,lay,large,spot,run,shock-corded,pole,appropriate,place,tent,tack,toss,rainfly,tack,camp,despite,big,i,effort,tear-down,very,easy,good,luck,back,bag,nice,-double,zipper,-,run,power,cord,large,gap,bug,-tub,floor,prevent,flooding,-plenty,room,2,adult,2,year-olds,air,mattress,if,'d,fit,i,'d,wager,fit,regular,king,size,mattress,i

In [11]:
tfidf = TfidfVectorizer()
tfs = tfidf.fit_transform(corpusA)

from sklearn.metrics.pairwise import cosine_similarity #Cosine sim calc
similarities = cosine_similarity(tfs)


In [12]:
print(similarities)

[[1.         0.         0.         ... 0.04504529 0.         0.0399029 ]
 [0.         1.         0.         ... 0.         0.10979112 0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.04504529 0.         0.         ... 1.         0.         0.03860585]
 [0.         0.10979112 0.         ... 0.         1.         0.        ]
 [0.0399029  0.         0.         ... 0.03860585 0.         1.        ]]


In [13]:

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpusA)
Xarr = X.toarray()
Xarr



array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
featurearr = vectorizer.get_feature_names_out()
featurearr = featurearr.tolist()

df = pd.DataFrame(data=Xarr)
df = pd.DataFrame(data=Xarr, columns=featurearr, index=df_b['asin'])
df


,00,000,0400,0500,06,0700,10,100,1000,10000,...,zap,zapping,zappy,zero,zika,ziploc,zipper,zipping,zone,zoom
asin,,,,,,,,,,,,,,,,,,,,,
B003U2UCTO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0018LA0UK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0012YUET0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00438RI7S,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00G8QGHL6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B00507AJ40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00IDDF7UE,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
B00Q46Z7A2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
row, col = df.shape

In [16]:


result = []
for i in range(0,1000):
  result.append(cs(0,i))

print(result)

[1.4291666666666665, 0.0, 0.0, 0, 0, 0, 0.08068715304598785, 0.0, 0.0, 0, 0.03952847075210474, 0.03315433736017064, 0.01656346649999844, 0.03952847075210474, 0.04057204129667897, 0.0, 0.020286020648339485, 0.03513641844631532, 0.009882117688026186, 0.0, 0.06944444444444443, 0.01571348402636772, 0.06454972243679027, 0.0, 0.027263762711732323, 0.04969039949999532, 0.04564354645876384, 0.0722222222222222, 0.11180339887498948, 0.11653431646335016, 0.022222222222222216, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07027283689263064, 0.0, 0.03513641844631532, 0.0, 0.0187812056606337, 0.1141933599505248, 0.0, 0.12909944487358055, 0, 0.0, 0.1141933599505248, 0.044194173824159216, 0, 0.01355815361366601, 0.06335435720647935, 0.0, 0.0, 0.0, 0.01571348402636772, 0, 0.0, 0.04969039949999532, 0.08451542547285164, 0.0, 0.012051692101036453, 0.0, 0.012051692101036453, 0, 0.0722222222222222, 0.0, 0.07905694150420949, 0.0, 0.0, 0.020286020648339485, 0, 0.04969039949999532, 0.01331536438971129, 0.018830768907869458, 0.0,

In [17]:
len(result)


1000

In [18]:
df_result = pd.DataFrame(data=result,columns=['CosSim'], index=df_b['asin'])
df_result = df_result[df_result.CosSim > 0]
df_result = df_result.sort_values(by=['CosSim'],ascending=False)

In [19]:
df_result

,CosSim
asin,
B003U2UCTO,1.429167
B00162OLTW,0.316228
B001T7QJ9O,0.273297
B00ROFX2NQ,0.211184
B01BA0QDBI,0.200000
...,...
B004AWOLV8,0.007217
B015TC7FN8,0.007217
B00JA82FV0,0.007217
